In [ ]:
import torch
from torch import nn
import numpy as np
from transformers import BertModel
from transformers import BertTokenizer

In [ ]:
class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased')
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 5) # CHANGE TO 2? / entropy classifier might also have to change
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [ ]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
softmax = nn.Softmax(dim=1)

In [ ]:
model_1 = BertClassifier()
model_1.load_state_dict(torch.load("model_2023_4_5.pth"))
model_1 = model_1.cuda()

In [ ]:
# Use the Model

text = "This is going to be an absolute disaster" # <-------- Input String

# input filtering (todo)
text = text.lower()
text = text.replace(".", "")
text = text.replace(",", "")
text = text.replace("-", "")
text = text.replace("`", "")
text = text.replace("'", "")
text = text.replace(": ", " ")
text = text.replace("; ", " ")
text = text.replace("&amp;", "and")
text = text.replace("&quot;", "and")
text = text.replace("?", "")
text = text.replace("!", "")
text = text.replace("*", "")
text = text.replace('"', "")
text = text.replace('’', "")
# -----------------------

token = tokenizer(text, padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
result = softmax(model_1(token["input_ids"].to(device), token["attention_mask"].reshape([1,1,512]).to(device)))

sentiment_negative = round(float(result[0][0]), 4)*100
sentiment_positive = round(float(result[0][4]), 4)*100

print("Positive Sentiment - ", round(float(result[0][-1])*100, 4), "%")
print("Negative Sentiment - ", round(float(result[0][0])*100, 4), "%")